In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px
import pandas as pd
from datetime import timedelta


In [28]:
transacciones = pd.read_csv("/Users/carlosoliver/Desktop/EDA_xnova_internacional/Entrega_EDA_Carlos_Oliver/US_SAMPLE.csv", sep= ",")

In [29]:
columns_to_select = ['consignee_address', 'product_desc', 'carrier_sasc_code', 'inbond_type',
                     'loading_port', 'unloading_port', 'container_type']

filtro = transacciones[columns_to_select].iloc[0:1,:]
filtro

,consignee_address,product_desc,carrier_sasc_code,inbond_type,loading_port,unloading_port,container_type
0,314 W SUPERIOR ST SUITE 201,CERAMIC TILES,"GNSI, GOUGH & SONS INC",0.0,"47537, LIVORNO","1703, SAVANNAH, GA",2210


In [30]:
transacciones.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129165 entries, 0 to 1129164
Data columns (total 35 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   complete_date          1129165 non-null  object 
 1   bill_of_lading         1129165 non-null  object 
 2   country                1128968 non-null  object 
 3   country_sure_level     1129165 non-null  int64  
 4   weight                 1129039 non-null  float64
 5   shipper_name           1129165 non-null  object 
 6   shipper_address        1129163 non-null  object 
 7   consignee_name         1129165 non-null  object 
 8   consignee_address      1129165 non-null  object 
 9   product_desc           1129154 non-null  object 
 10  hs_code                1129165 non-null  int64  
 11  hs_code_sure_level     1129165 non-null  int64  
 12  estimate_arrival_date  1129165 non-null  int64  
 13  carrier_sasc_code      1129165 non-null  object 
 14  vessel_country_cod

In [31]:
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import joblib
import xgboost as xgb
from datetime import timedelta

# Suprimir los warnings
warnings.filterwarnings("ignore")


In [32]:

# Convertir columnas de fecha y calcular el tiempo de retraso
transacciones["estimate_arrival_date"] = pd.to_datetime(transacciones["estimate_arrival_date"].astype(str), format='%Y%m%d')
transacciones["complete_date"] = pd.to_datetime(transacciones["complete_date"])
transacciones['delay_time'] = transacciones['complete_date'] - transacciones['estimate_arrival_date']

# Define el rango razonable para el tiempo de retraso (en días)
min_delay = timedelta(days=1)
max_delay = timedelta(days=30)

# Filtra los datos para eliminar valores atípicos
df = transacciones[(transacciones['delay_time'] >= min_delay) & (transacciones['delay_time'] <= max_delay)]

# Eliminar columnas innecesarias
df_clean = df.drop(['bill_of_lading', 'country_sure_level', 'marks_numbers', 'true_supplier', 'true_buyer', 'manifest_n', 'day', 'states', 
                    'year', 'hs_code_sure_level', 'hs_code', 'container_id', 'quantity_unit', 'notify_party_name', 'voyage', 
                    'vessel_country_code', 'vessel_code', 'quantity', 'place_of_receipt', 'vessel_name', 'consignee_name', 
                    'shipper_address', 'shipper_name', 'weight', 'country', 'month', 'product_desc', 'inbond_type'], axis=1)
pd.options.display.max_columns = None

df_clean = df_clean.head(2000)


In [33]:

# Función para categorizar el retraso
def categorize_delay(days):
    if days <= timedelta(days=10):
        return 0
    elif days <= timedelta(days=20):
        return 1
    else:
        return 2


In [34]:

# Aplicar la función de categorización y eliminar columnas innecesarias
df_clean['delay_category'] = df_clean['delay_time'].apply(categorize_delay)
df_clean.drop(['delay_time', 'complete_date'], axis=1, inplace=True)

# Convertir columnas de fecha en características numéricas
df_clean['estimate_arrival_date_year'] = df_clean['estimate_arrival_date'].dt.year
df_clean['estimate_arrival_date_month'] = df_clean['estimate_arrival_date'].dt.month
df_clean['estimate_arrival_date_day'] = df_clean['estimate_arrival_date'].dt.day
df_clean.drop(['estimate_arrival_date'], axis=1, inplace=True)

# Separar las características de la variable objetivo
X = df_clean.drop(['delay_category'], axis=1)
y = df_clean['delay_category']


In [35]:
X

,consignee_address,carrier_sasc_code,loading_port,unloading_port,container_type,estimate_arrival_date_year,estimate_arrival_date_month,estimate_arrival_date_day
0,314 W SUPERIOR ST SUITE 201,"GNSI, GOUGH & SONS INC","47537, LIVORNO","1703, SAVANNAH, GA",2210,2019,2,24
2,200 WEST SAND LAKE ROAD SUITE 800,OPLT,"58309, KAO HSIUNG","5201, MIAMI, FL",42G0,2019,12,13
4,7555 BEEDIE WAY DELTA BC V4G0A5 CA,"SHKK, SCHOOKS TRUCKING","57035, SHANGHAI","3001, SEATTLE, WA",45G1,2019,12,15
6,191 PARKWAY WEST DUNCAN SC 29334 US,"BANQ, KUEHNE & NAGEL INC.","42870, BREMERHAVEN","1601, CHARLESTON, SC",4510,2019,10,23
7,C O L1M SUPPLY INC 1648 NW 34TH TERRACE,OPLT,"58309, KAO HSIUNG","5201, MIAMI, FL",0,2019,11,29
...,...,...,...,...,...,...,...,...
3145,530 7TH AVE NEW YORK N Y 10018 USA,"APLU, AMERICAN PRESIDENT LINES LTD","57035, SHANGHAI","2704, LOS ANGELES, CA",4205,2020,1,6
3147,425 106TH AVE NE BELLEVUE WA 98004 USA BELLEVU...,APVF,"58309, KAO HSIUNG","3001, SEATTLE, WA",22G0,2019,12,27
3148,6341 TUCKAWAY COVE AVE LAS VEGAS LAS VEGAS 891...,APXL,"55206, VUNG TAU","2709, LONG BEACH, CA",LEG0,2020,1,16
3149,0,"ASFN, ATLANTA SPECIALTY FOOD DISTRIBUTORS","55206, VUNG TAU","1703, SAVANNAH, GA","45G0,45G0,45G0",2020,1,16


In [38]:
import pandas as pd
import numpy as np
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Supongamos que tienes X (datos) y y (etiquetas)
# Define las columnas categóricas y numéricas
categorical_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
X_encoded = label_encoder.fit_transform(X)


ValueError: y should be a 1d array, got an array of shape (2000, 8) instead.

In [37]:

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded , y, test_size=0.2, random_state=42)

# Configurar el pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

# Definir los modelos y parámetros para GridSearch
param_grid = [
    {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [3, 6],
        'model__learning_rate': [0.01, 0.1, 0.2]
    },
]

# Configurar y ejecutar el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y los mejores parámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejor modelo:", best_model)
print("Mejores parámetros:", best_params)
print("Mejor Accuracy Score en validación cruzada:", best_score)

# Guardar el mejor modelo y el label encoder en el mismo archivo pickle
joblib.dump({
    'model': best_model,
    'label_encoder': label_encoder
}, 'best_model_with_preprocessing.pkl')

# Evaluar el modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
test_accuracy_score = accuracy_score(y_test, y_pred)
print("Accuracy Score en el conjunto de test:", test_accuracy_score)



NameError: name 'X_encoded' is not defined